In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import cv2
import supervision as sv
from osgeo import gdal, osr
from scipy.spatial import cKDTree
from typing import List, Tuple
from shapely.geometry import Point, Polygon

In [2]:
from utils import (
    process_geotif_single_tile,
    process_geotif_with_sam_tiled,
    filter_car_polygons_aerial,
    save_features_to_shapefile,
    visualize_filtered_cars,
    visualize_detections_on_image,
    compare_geotif_outputs,
)

c:\Users\david.bruner\.conda\envs\geoaienv\Lib\site-packages\pyproj\__init__.py:95: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [3]:
HOME = os.getcwd()
CHECKPOINT_PATH = os.path.join(HOME, "sam_weights", "sam_vit_h_4b8939.pth")
IMAGE_2016_PATH = "image_data/auckland-0075m-urban-aerial-photos-2015-2016.tif"
IMAGE_2017_PATH = "image_data/auckland-0075m-urban-aerial-photos-2017.tif"


In [4]:
all_car_polygons_2016, all_car_centroids_2016, dataset_2016, projection_2016 = process_geotif_with_sam_tiled(IMAGE_2016_PATH, CHECKPOINT_PATH, "whole_akl_2016_test1")


c:\Users\david.bruner\.conda\envs\geoaienv\Lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Using device: cuda:0
Image size: 4859 2818
Number of bands: 4
Number of tiles: 6 x 4 = 24


Processing tiles:   0%|          | 0/24 [00:00<?, ?tile/s]

Tile 1/24 processed in 7.60 seconds
Tile 2/24 processed in 7.41 seconds
Tile 3/24 processed in 7.68 seconds
Tile 4/24 processed in 7.78 seconds
Tile 5/24 processed in 7.64 seconds
Tile 6/24 processed in 7.39 seconds
Tile 7/24 processed in 7.58 seconds
Tile 8/24 processed in 8.18 seconds
Tile 9/24 processed in 8.13 seconds
Tile 10/24 processed in 8.10 seconds
Tile 11/24 processed in 7.94 seconds
Tile 12/24 processed in 7.85 seconds
Tile 13/24 processed in 7.93 seconds
Tile 14/24 processed in 8.26 seconds
Tile 15/24 processed in 8.36 seconds
Tile 16/24 processed in 8.00 seconds
Tile 17/24 processed in 7.77 seconds
Tile 18/24 processed in 7.51 seconds
Tile 19/24 processed in 7.70 seconds
Tile 20/24 processed in 7.83 seconds
Tile 21/24 processed in 7.87 seconds
Tile 22/24 processed in 7.27 seconds
Tile 23/24 processed in 7.55 seconds
Tile 24/24 processed in 7.17 seconds
GeoDataFrame saved to image_data/whole_akl_2016_test1_all_polygons


c:\Users\david.bruner\.conda\envs\geoaienv\Lib\site-packages\pyogrio\core.py:27: RuntimeWarning: Could not detect PROJ data files.  Set PROJ_LIB environment variable to the correct path.
  _init_proj_data()


GeoDataFrame saved to image_data/whole_akl_2016_test1_car_polygons
GeoDataFrame saved to image_data/whole_akl_2016_test1_car_centroids


In [5]:
all_car_polygons_2017, all_car_centroids_2017, dataset_2017, projection_2017 = process_geotif_with_sam_tiled(IMAGE_2017_PATH, CHECKPOINT_PATH, "whole_akl_2017_test1")

c:\Users\david.bruner\.conda\envs\geoaienv\Lib\site-packages\osgeo\gdal.py:4216: RuntimeWarning: PROJ: proj_create_from_database: C:\Program Files\PostgreSQL\13\share\contrib\postgis-3.4\proj\proj.db contains DATABASE.LAYOUT.VERSION.MINOR = 2 whereas a number >= 4 is expected. It comes from another PROJ installation.
  return _gdal.Dataset_GetProjection(self, *args)
c:\Users\david.bruner\.conda\envs\geoaienv\Lib\site-packages\osgeo\gdal.py:4216: RuntimeWarning: The definition of projected CRS EPSG:2193 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
  return _gdal.Dataset_GetProjection(self, *args)


Using device: cuda:0
Image size: 4859 2818
Number of bands: 4
Number of tiles: 6 x 4 = 24


Processing tiles:   0%|          | 0/24 [00:00<?, ?tile/s]

Tile 1/24 processed in 7.41 seconds
Tile 2/24 processed in 7.56 seconds
Tile 3/24 processed in 7.73 seconds
Tile 4/24 processed in 7.90 seconds
Tile 5/24 processed in 7.99 seconds
Tile 6/24 processed in 7.49 seconds
Tile 7/24 processed in 7.76 seconds
Tile 8/24 processed in 7.91 seconds
Tile 9/24 processed in 8.17 seconds
Tile 10/24 processed in 8.04 seconds
Tile 11/24 processed in 8.08 seconds
Tile 12/24 processed in 7.73 seconds
Tile 13/24 processed in 7.77 seconds
Tile 14/24 processed in 7.83 seconds
Tile 15/24 processed in 8.21 seconds
Tile 16/24 processed in 7.93 seconds
Tile 17/24 processed in 7.82 seconds
Tile 18/24 processed in 7.48 seconds
Tile 19/24 processed in 7.44 seconds
Tile 20/24 processed in 7.42 seconds
Tile 21/24 processed in 7.83 seconds
Tile 22/24 processed in 7.39 seconds
Tile 23/24 processed in 7.21 seconds
Tile 24/24 processed in 7.53 seconds
GeoDataFrame saved to image_data/whole_akl_2017_test1_all_polygons
GeoDataFrame saved to image_data/whole_akl_2017_test1_

In [8]:
match_results = compare_geotif_outputs(all_car_polygons_2016, all_car_centroids_2016, all_car_polygons_2017, all_car_centroids_2017, 1.0)
print(match_results['unmatched_polygons1'])

[<POLYGON ((1752789.3 5933858.925, 1752789.225 5933858.85, 1752788.925 593385...>, <POLYGON ((1752785.7 5933829.3, 1752785.475 5933829.075, 1752785.475 5933829...>, <POLYGON ((1752796.8 5933854.275, 1752796.65 5933854.125, 1752796.65 5933854...>, <POLYGON ((1752776.025 5933835.9, 1752775.95 5933835.825, 1752775.875 593383...>, <POLYGON ((1752780.6 5933842.2, 1752780.525 5933842.125, 1752780.45 5933842....>, <POLYGON ((1752782.775 5933825.85, 1752782.55 5933825.625, 1752782.55 593382...>, <POLYGON ((1752846.9 5933849.025, 1752846.825 5933848.95, 1752846.675 593384...>, <POLYGON ((1752787.2 5933862.9, 1752787.125 5933862.825, 1752787.05 5933862....>, <POLYGON ((1752778.2 5933818.725, 1752778.125 5933818.65, 1752777.9 5933818....>, <POLYGON ((1752783.45 5933858.55, 1752783.375 5933858.475, 1752783.3 5933858...>, <POLYGON ((1752776.775 5933858.55, 1752776.7 5933858.475, 1752776.625 593385...>, <POLYGON ((1752776.625 5933858.475, 1752776.55 5933858.4, 1752776.475 593385...>, <POLYGON ((1752

In [11]:
save_features_to_shapefile(match_results['matched_polygons_1'], projection_2016, "whole_akl_2016_test1_matched")

GeoDataFrame saved to whole_akl_2016_test1_matched


In [12]:
save_features_to_shapefile(match_results['matched_polygons_2'], projection_2017, "whole_akl_2017_test1_matched")

GeoDataFrame saved to whole_akl_2017_test1_matched
